In [1]:
import foursquare
import json
import io
import pymongo
import sys

In [3]:
CLIENT_ID = 'PNCM0L4KGSFQPB3NB43CARKGKVE1EHHTGBHKI5BETD1MDRDB'
CLIENT_SECRET = 'ZJJWOM5EFSDU3Z3ULQDV5PE4CJMEMCCEPMTUVQI5WALNSQIC'

# Construct the client object
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, version='20140806')

In [4]:
# Build the authorization url for your app
auth_uri = client.oauth.auth_url()

print client
print auth_uri

https://foursquare.com/oauth2/authenticate?redirect_uri=None&response_type=code&client_id=PNCM0L4KGSFQPB3NB43CARKGKVE1EHHTGBHKI5BETD1MDRDB


In [5]:
# Get auth token
access_token = client.oauth.get_token('WHIP2BCNXQRMAVYCRUYHSTYJWQMO4LYKLIBEJU2GGY3RP0L5')
client.set_access_token(access_token)

print access_token

M0TXH1FKXXZ52ZXV3ZNCYLHCUSF55B0GJZITAN15FDCJLCWQ


In [6]:
def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    
    # Connects to the MongoDB server surring on
    # localhost:27017 by default
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    
    # Get a reference to a particular database
    db = client[mongo_db]
    
    # Reference a particular collection in the database
    coll = db[mongo_db_coll]
    
    # Perform a bulk insert and return the IDs
    return coll.insert_one(data)

def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False, criteria=None, projection=None, **mongo_conn_kw):
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    db = client[mongo_db]
    coll = db[mongo_db_coll]
    
    if criteria is None:
        criteria = {}
        
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)
        
    # Returning a cursor is recommended for large amounts of data
    if return_cursor:
        return cursor
    else:
        return [item for iten in cursor]
    
def save_json(filename, data):
    with io.open('resources/foursquare/{0}.json'.format(filename),
                'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(data, ensure_ascii=False)))
        
def load_json(filename):
    with io.open('resources/foursquare/{0}.json'.format(filename),
                    encoding='utf-8') as f:
        return f.read()

mongo = pymongo.MongoClient()
db = mongo["mean-dev"]

In [40]:
height = 0
max_height = 1
current_user = 0

all_friends = []
all_friends_nb = []

me = client.users()['user']
my_friends = client.users.friends()['friends']['items']
current_user_friends = []

ss_f = {}
ss_f['firstName'] = me['firstName']
ss_f['lastName'] = me['lastName']
ss_f['homeCity'] = me['homeCity']
ss_f['gender'] = me['gender']
ss_f['bio'] = me['bio']
ss_f['email'] = me['contact']['email'] if me['contact'].has_key('email') else me['lastName'] + me['firstName'] + '.' + me['gender'] + '@gmail.com'
ss_f['phone'] = me['contact']['phone'] if me['contact'].has_key('phone') else None

ss_f['contact'] = {}
ss_f['contact']['facebook'] = me['contact']['facebook'] if me['contact'].has_key('facebook') else None
ss_f['contact']['twitter'] = me['contact']['twitter'] if me['contact'].has_key('twitter') else None
ss_f['contact']['foursquare'] = me['id']

# coll = db["users"]
if db.users.find_one({ 'email' : ss_f['email']}) is None:
    save_to_mongo(ss_f, "mean-dev", "users")

    del ss_f['_id']
ss_f['id'] = me['id']
ss_f['contact']['email'] = me['contact']['email'] if me['contact'].has_key('email') else me['lastName'] + me['firstName'] + '.' + me['gender'] + '@gmail.com'
ss_f['contact']['phone'] = me['contact']['phone'] if me['contact'].has_key('phone') else None

del ss_f['email']
del ss_f['phone']
del ss_f['contact']['foursquare']
save_json(me['id'], ss_f)

for friend in my_friends:
    ss_f = {}
    ss_f['firstName'] = friend['firstName']
    ss_f['lastName'] = friend['lastName'] if friend.has_key('lastName') else None
    ss_f['homeCity'] = friend['homeCity']
    ss_f['gender'] = friend['gender']
    ss_f['bio'] = friend['bio']
    ss_f['email'] = friend['contact']['email'] if friend['contact'].has_key('email') else friend['lastName'] + friend['firstName'] + '.' + friend['gender'] + '@gmail.com'
    ss_f['phone'] = friend['contact']['phone'] if friend['contact'].has_key('phone') else None

    ss_f['contact'] = {}
    ss_f['contact']['facebook'] = friend['contact']['facebook'] if friend['contact'].has_key('facebook') else None
    ss_f['contact']['twitter'] = friend['contact']['twitter'] if friend['contact'].has_key('twitter') else None
    ss_f['contact']['foursquare'] = friend['id']
    
    if db.users.find_one({ 'email' : ss_f['email']}) is None:
        save_to_mongo(ss_f, "mean-dev", "users")
        
        del ss_f['_id']
        
    
    ss_f['id'] = friend['id']
    ss_f['contact']['email'] = friend['contact']['email'] if friend['contact'].has_key('email') else friend['lastName'] + friend['firstName'] + '.' + friend['gender'] + '@gmail.com'
    ss_f['contact']['phone'] = friend['contact']['phone'] if friend['contact'].has_key('phone') else None

    del ss_f['email']
    del ss_f['phone']
    del ss_f['contact']['foursquare']
    current_user_friends.append(ss_f)

all_friends.extend(current_user_friends)
all_friends_nb.append(len(current_user_friends))

filename = me['id'] + '.friend'
print "\n" + filename + "\n"
save_json(filename, current_user_friends)

height = height + 1
# current_user = current_user + 1


77562607.friend



In [41]:
height = 1
current_user = 0
while height <= max_height:
    
    if (height == 1):
        nb_users = all_friends_nb[current_user]
    else: 
        nb_users = sum([all_friends_nb[current_user + i + 1] for i in range(nb_users) ])
    
    print str(nb_users)
    print
    print
    
    for i in range(nb_users):
        user = all_friends[current_user + i + 1]
        
        friends = client.users.friends(user['id'])['friends']['items']
        save_json(friend['id'], friend)
        
        current_user_friends = []
        for friend in friends: # user['user']['friends']['groups'][1]['items']:
            ss_f = {}
            ss_f['firstName'] = friend['firstName']
            ss_f['lastName'] = friend['lastName'] if friend.has_key('lastName') else None
            ss_f['homeCity'] = friend['homeCity']
            ss_f['gender'] = friend['gender']
            ss_f['bio'] = friend['bio']
            ss_f['email'] = friend['contact']['email'] if friend['contact'].has_key('email') else friend['firstName'] + '.' + friend['gender'] + '@gmail.com'
            ss_f['phone'] = friend['contact']['phone'] if friend['contact'].has_key('phone') else None

            ss_f['contact'] = {}
            ss_f['contact']['facebook'] = friend['contact']['facebook'] if friend['contact'].has_key('facebook') else None
            ss_f['contact']['twitter'] = friend['contact']['twitter'] if friend['contact'].has_key('twitter') else None
            ss_f['contact']['foursquare'] = friend['id']

            
            # mongo = pymongo.MongoClient()
            # db = mongo["mean-dev"]
            # coll = db["users"]
            if db.users.find_one({ 'email' : ss_f['email']}) is None:
                save_to_mongo(ss_f, "mean-dev", "users")

                del ss_f['_id']
            else:
                print 
                print 'USER IS IN'
                print ss_f
                print
            
            ss_f['id'] = friend['id']
            ss_f['contact']['email'] = friend['contact']['email'] if friend['contact'].has_key('email') else friend['firstName'] + '.' + friend['gender'] + '@gmail.com'
            ss_f['contact']['phone'] = friend['contact']['phone'] if friend['contact'].has_key('phone') else None

            del ss_f['email']
            del ss_f['phone']
            del ss_f['contact']['foursquare']
            current_user_friends.append(ss_f)
        
        print "Length current_user_friends"
        print len(current_user_friends)
        filename = user['id'] + '.friend'
        save_json(filename, current_user_friends)
        
        all_friends.extend(current_user_friends)
        all_friends_nb.append(len(current_user_friends))
        
    height = height + 1
    current_user = current_user + nb_users

19



USER IS IN
{'bio': u'', 'firstName': u'Cynthe', 'lastName': u'Ibohn', 'homeCity': u'Douala, Cameroon', 'phone': None, 'contact': {'twitter': u'ibohn2', 'foursquare': u'74010682', 'facebook': u'706804821'}, 'gender': u'female', 'email': u'myde001@yahoo.fr'}


USER IS IN
{'bio': u'', 'firstName': u'G-rar', 'lastName': u"F'ce", 'homeCity': u'Douala, Cameroon', 'phone': None, 'contact': {'twitter': u'grarfce', 'foursquare': u'81326524', 'facebook': u'10206454701220112'}, 'gender': u'male', 'email': u'nkwetche2000@yahoo.fr'}


USER IS IN
{'bio': u'', 'firstName': u'Mael', 'lastName': u'FOSSO', 'homeCity': u'', 'phone': u'+23795165033', 'contact': {'twitter': u'sivleam', 'foursquare': u'77562607', 'facebook': None}, 'gender': u'male', 'email': u'fosso.mael.elvis@gmail.com'}


USER IS IN
{'bio': u'', 'firstName': u'larira', 'lastName': u'a', 'homeCity': u'', 'phone': None, 'contact': {'twitter': u'lariraa88', 'foursquare': u'96506957', 'facebook': None}, 'gender': u'female', 'email': u'

In [70]:
me = db.users.find_one({'email': 'fosso.mael.elvis@gmail.com'})
# venues_history = client.users.venuehistory()['venues']['items'];
venues = client.venues.search({ 'near':'Yaounde','intent':'browse'})
print len(venues['venues'])
print
print
for venue in venues['venues']:
    place = {}
    # venue = venue['venue']
    
    place['name'] = venue['name']
    place['verified'] = venue['verified']
    
    place['contact'] = {}
    place['contact']['phone'] = venue['contact']['phone'] if venue['contact'].has_key('phone') else None
    place['contact']['facebook'] = venue['contact']['facebook'] if venue['contact'].has_key('facebook') else None
    place['contact']['facebookName'] = venue['contact']['facebookName'] if venue['contact'].has_key('facebookName') else None
    place['contact']['twitter'] = venue['contact']['twitter'] if venue['contact'].has_key('twitter') else None
    place['contact']['foursquare'] = venue['id']
    
    place['location'] = {}    
    place['location']['address'] = venue['location']['address'] if venue['location'].has_key('address') else None
    place['location']['lat'] = venue['location']['lat']
    place['location']['lng'] = venue['location']['lng']
    place['location']['cc'] = venue['location']['cc']
    place['location']['city'] = venue['location']['city'] if venue['location'].has_key('city') else None
    place['location']['state'] = venue['location']['state'] if venue['location'].has_key('state') else None
    place['location']['country'] = venue['location']['country']
    place['location']['formattedAddresss'] = venue['location']['formattedAddress'] if venue['location'].has_key('formattedAddress') else None
    
    place['categories'] = []
    for categorie in venue['categories']:
        cat = {}
        cat['name'] = categorie['name']
        cat['shortName'] = categorie['shortName']
        cat['foursquare'] = categorie['id']
        
        place['categories'].append(cat)
    
    if (db.places.find_one({'name':place['name']}) is None):
        # db.venues.insert_one(place)
        db.places.insert_one(place)
        del place['_id']
        
    print json.dumps(venue, indent=4)
    print
    print
    print


30


{
    "verified": true, 
    "name": "Hilton Yaounde", 
    "referralId": "v-1472104243", 
    "venueChains": [
        {
            "id": "556e146fa7c82e6b725097d5"
        }
    ], 
    "url": "http://www.placeshilton.com/yaounde-cameroon", 
    "storeId": "YAOHITW", 
    "specials": {
        "count": 0, 
        "items": []
    }, 
    "allowMenuUrlEdit": true, 
    "contact": {
        "phone": "22233646", 
        "twitter": "hiltonhotels", 
        "facebook": "137619719644857", 
        "formattedPhone": "22233646", 
        "facebookName": "Hilton Bonn"
    }, 
    "hasPerk": false, 
    "location": {
        "city": "Yaound\u00e9", 
        "labeledLatLngs": [
            {
                "lat": 3.8643980543805476, 
                "lng": 11.515950894947037, 
                "label": "display"
            }
        ], 
        "cc": "CM", 
        "country": "Cameroon", 
        "postalCode": "11852", 
        "state": "Center", 
        "formattedAddress": [
         

In [69]:
checkins = client.users.checkins()['checkins']['items']
print checkins
print
print
for checkin in checkins:
    venue = checkin['venue']
    
    print json.dumps(venue, indent=4)
    print
    
    place['name'] = venue['name']
    place['verified'] = venue['verified']
    
    place['contact'] = {}
    place['contact']['phone'] = venue['contact']['phone'] if venue['contact'].has_key('phone') else None
    place['contact']['facebook'] = venue['contact']['facebook'] if venue['contact'].has_key('facebook') else None
    place['contact']['facebookName'] = venue['contact']['facebookName'] if venue['contact'].has_key('facebookName') else None
    place['contact']['twitter'] = venue['contact']['twitter'] if venue['contact'].has_key('twitter') else None
    place['contact']['foursquare'] = venue['id']
    
    place['location'] = {}    
    place['location']['address'] = venue['location']['address'] if venue['location'].has_key('address') else None
    place['location']['lat'] = venue['location']['lat']
    place['location']['lng'] = venue['location']['lng']
    place['location']['cc'] = venue['location']['cc']
    place['location']['city'] = venue['location']['city'] if venue['location'].has_key('city') else None
    place['location']['state'] = venue['location']['state'] if venue['location'].has_key('state') else None
    place['location']['country'] = venue['location']['country']
    place['location']['formattedAddresss'] = venue['location']['formattedAddress'] if venue['location'].has_key('formattedAddress') else None
    
    place['categories'] = []
    for categorie in venue['categories']:
        cat = {}
        cat['name'] = categorie['name']
        cat['shortName'] = categorie['shortName']
        cat['foursquare'] = categorie['id']
        
        place['categories'].append(cat)
    
    if (db.places.find_one({'name':place['name']}) is None):
        db.places.insert_one(place)
        del place['_id']
    

[{u'like': False, u'isMayor': True, u'source': {u'url': u'https://www.swarmapp.com', u'name': u'Swarm for Android'}, u'posts': {u'count': 0, u'textCount': 0}, u'venue': {u'verified': False, u'name': u'Centre Pasteur du Cameroun', u'contact': {}, u'location': {u'lat': 3.87810615432176, u'cc': u'CM', u'lng': 11.512132891790314, u'labeledLatLngs': [{u'lat': 3.87810615432176, u'lng': 11.512132891790314, u'label': u'display'}], u'country': u'Cameroon'}, u'stats': {u'tipCount': 0, u'checkinsCount': 31, u'usersCount': 16}, u'id': u'528b454411d2016fe4a3f466', u'categories': [{u'pluralName': u'Medical Labs', u'primary': True, u'name': u'Medical Lab', u'shortName': u'Medical Lab', u'id': u'4f4531b14b9074f6e4fb0103', u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/education/lab_', u'suffix': u'.png'}}], u'like': False}, u'comments': {u'count': 0}, u'photos': {u'count': 0, u'items': []}, u'createdAt': 1471684918, u'type': u'checkin', u'id': u'57b82136498eee4e9c606da8', u'timeZoneOffs

In [16]:
venuelikes = client.users.venuelikes()
# print json.dumps(venuelikes)
for venue in venuelikes['venues']['items']:
    print json.dumps(venue, indent=4)
    print
    print


{
    "verified": false, 
    "name": "Safari Night Club", 
    "tipHint": "People talk about good for groups, full bar, happy hour.", 
    "ratedAt": 1472364384, 
    "allowMenuUrlEdit": true, 
    "contact": {
        "phone": "22501872", 
        "formattedPhone": "22501872"
    }, 
    "location": {
        "city": "Yaound\u00e9", 
        "labeledLatLngs": [
            {
                "lat": 3.874530819890243, 
                "lng": 11.518306732177734, 
                "label": "display"
            }
        ], 
        "cc": "CM", 
        "country": "Cameroon", 
        "postalCode": "1000", 
        "state": "Centre", 
        "formattedAddress": [
            "avenue churchill", 
            "Yaound\u00e9 1000"
        ], 
        "address": "avenue churchill", 
        "lat": 3.874530819890243, 
        "lng": 11.518306732177734
    }, 
    "stats": {
        "tipCount": 9, 
        "checkinsCount": 270, 
        "usersCount": 98
    }, 
    "id": "4f338e1dbb3d0db196e794